In [1]:
import os
import re
import json
import glob
from collections import defaultdict
from functools import partial

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
#from tqdm.autonotebook import tqdm

import nltk
import string
from fuzzywuzzy import fuzz

from model import longest_consecutive_caps as LCC
from model import KMP
# from model.model import longest_consecutive_caps as LCC
#from model.model import KMP


# Spacy model
import spacy
#from __future__ import unicode_literals, print_function
#import plac
import random
from pathlib import Path
from tqdm import tqdm
from spacy.training.example import Example


output_dir="output/"
model = None 
#model = "specified" # specified for transformer + ner only
n_iter = 10 # number of training iteration

/home/ry/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#!python -m spacy download en_core_web_trf

In [3]:
train_df = pd.read_csv("dataset/train.csv")
sample_sub = pd.read_csv('dataset/sample_submission.csv')
train_fp = "dataset/train/"
test_fp = "dataset/test/"

# train_df = pd.read_csv("/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv")
# sample_sub = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
# train_fp = "/kaggle/input/coleridgeinitiative-show-us-the-data/train/"
# test_fp = "/kaggle/input/coleridgeinitiative-show-us-the-data/test/"

In [4]:
#train_df.head(5)

In [5]:
#train_df.info()

In [6]:
def read_append_return(filename, train_files_path=train_fp, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [7]:
%%time
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

/home/ry/.local/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 19661/19661 [00:05<00:00, 3810.47it/s]

CPU times: user 4.34 s, sys: 845 ms, total: 5.18 s
Wall time: 5.17 s


In [8]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = re.sub(' +', ' ', str(text).lower()).strip()
    #text = ''.join([k for k in text if k not in string.punctuation])
    #text = re.sub('[^A-Za-z0-9\(\)]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    return text

In [9]:
%%time
tqdm.pandas()
train_df['text'] = train_df['text'].progress_apply(text_cleaning)

100%|██████████| 19661/19661 [00:57<00:00, 339.88it/s]

CPU times: user 55.5 s, sys: 2.65 s, total: 58.2 s
Wall time: 57.9 s


In [10]:
a = train_df["Id"].nunique()
b = train_df["Id"].size
print(a, b)
print(a/b)

14316 19661
0.7281420070189716


```
from model import search_sentences
from custom_classes import PosMap
```

```
%%time
tqdm.pandas()

i = 0
limit = 100
p = 5
q = 4
pre_tf = PosMap(p)
post_tf = PosMap(q)

for row in tqdm(train_df.iterrows()):
    label = row[1]["cleaned_label"]
    text = row[1]["text"].lower()
    act, deact = search_sentences(label, text, pre=p, post=q)

    for j in range(len(act)):
        pre_words = act[j].split()[::-1]
        post_words = deact[j].split()
        for k in range(min([len(pre_words), p])):
            try:
                word = pre_words[k]
                pre_tf[k][word] += 1
            except IndexError:
                print(act[j])
        
        for k in range(min([len(post_words), q])):
            try:
                word = post_words[k]
                post_tf[k][word] += 1
            except IndexError:
                print(deact[j])

    i += 1
    #if i >= limit: break
```

```
pre_tf.plot(idx=0) # the first closest pre words
pre_tf.plot(idx=1) # the second closest pre words
pre_tf.plot(idx=2) # the third
pre_tf.plot(idx=3)
```

```
post_tf.plot(idx=0)
post_tf.plot(idx=1)
post_tf.plot(idx=2)
```

In [11]:
#train_df.head()

In [12]:
%%time
tqdm.pandas()
sample_sub['text'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_fp))

100%|██████████| 4/4 [00:00<00:00, 1380.50it/s]

CPU times: user 6.93 ms, sys: 0 ns, total: 6.93 ms
Wall time: 5.71 ms


In [13]:
def clean_text(txt):
    ''' DO NOT DELETE: Official function for submission text cleaning '''
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()
    #return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [14]:
# train data prep
TRAIN_DATA = []
if model == "specified":
    token_anno = "entities"
else:
    token_anno = "entities"


for index, row in tqdm(train_df.iterrows()):
    # get text of each sample test
    train_text = row['text']
    row_id = row['Id']
    label = row['cleaned_label']
    #text = sample_text.lower().split('.')
    text = re.split('[?.,;\n\t&!()]+', train_text) # can't have sample_text.lower() since I need to find consecutive caps

    # begin matching
    for sentence in text:
        # clean text 
        sentence = clean_text(sentence)
        indexed = KMP(label, sentence)
        if indexed != []:
            elist = []
            for i in indexed:
                entity = (i, len(label)+i, "DATASET") 
                elist.append(entity)
            dic = {token_anno:elist}
            x = (sentence, dic)
            TRAIN_DATA.append(x)

19661it [08:02, 40.73it/s] 


In [15]:
#load the model
print(spacy.require_gpu())

if model == "specified":
    nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
elif model == "generic":
    nlp = spacy.load("en_core_web_sm")
elif model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")


#set up the pipeline
if 'ner' not in nlp.pipe_names:
    #ner = nlp.create_pipe('ner') # version < 3.0
    #nlp.add_pipe(ner, last=True) # verions < 3.0
    nlp.add_pipe("ner", last=True) # version >= 3.0 only
    ner = nlp.get_pipe("ner")
else:
    ner = nlp.get_pipe('ner')

#for _, annotations in TRAIN_DATA:
#    for ent in annotations.get('entities'):
#        ner.add_label(ent[2])
ner.add_label("DATASET")


other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    if model == None: 
        optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],
                drop=0.5,  
                losses=losses)
            #nlp.update(
            #    [text],
            #    [annotations],
            #    drop=0.5,  
            #    losses=losses)
        print(losses)

True
Created blank 'en' model


  0%|          | 20/61735 [00:00<35:37, 28.87it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "adniinfo" with entities "[(0, 4, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  0%|          | 25/61735 [00:00<32:06, 32.03it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "6 percentage distribution of 1989 90 beginning pos..." with entities "[(37, 68, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  0%|          | 45/61735 [00:01<32:49, 31.32it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_u

  1%|          | 336/61735 [00:10<33:16, 30.75it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "and the adjusted percentage after taking into acco..." with entities "[(143, 174, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  1%|          | 340/61735 [00:10<34:34, 29.59it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "in adni2 study" with entities "[(3, 7, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  1%|          | 372/61735 [00:11<31:51, 32.10it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/tra

  1%|          | 733/61735 [00:22<33:02, 30.77it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "org scientists adnistudyprocedures" with entities "[(15, 19, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  1%|          | 741/61735 [00:23<31:11, 32.59it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "and adni2" with entities "[(4, 8, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  1%|          | 749/61735 [00:23<32:02, 31.73it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: Us

  2%|▏         | 1031/61735 [00:32<30:55, 32.71it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "5a percentage distribution of 1995 96 beginning po..." with entities "[(38, 69, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  2%|▏         | 1043/61735 [00:32<32:19, 31.30it/s]/home/ry/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "edu wp content uploads 2008 07 adni2 procedures ma..." with entities "[(31, 35, 'DATASET')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  2%|▏         | 1169/61735 [00:36<31:12, 32.34it/s]/home/ry/anaco

...

{'ner': 8123.722051320774}


  0%|          | 3/61735 [00:00<36:36, 28.11it/s]

{'ner': 8056.539387324536}


100%|██████████| 61735/61735 [32:50<00:00, 31.33it/s]

{'ner': 8173.5872163282975}


In [16]:
# sample test
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    break

# save model
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Entities [('adni', 'DATASET')]
Saved model to output


In [17]:
# STRING MATCHING BLOCK
temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)
id_list = []
lables_list = []
# load model 'EN'
nlp2 = spacy.load(output_dir)
for index, row in tqdm(sample_sub.iterrows()):
    # get text of each sample test
    sample_text = row['text']
    row_id = row['Id']
    temp_df = train_df[train_df['text'] == text_cleaning(sample_text)]

    cleaned_labels = temp_df['cleaned_label'].to_list()

    #text = sample_text.lower().split('.')
    text = re.split('[?.,;\n\t&!]', sample_text) # can't have sample_text.lower() since I need to find consecutive caps

    # begin search
    # matching
    
    for known_label in existing_labels:   # for each label in the known set
        # EXACT MATCH
        if known_label in sample_text.lower():   # find the EXACT label in text 
            cleaned_labels.append(clean_text(known_label)) # if found, then append to the list for further formatting
            
    for sentence in text:
        doc = nlp2(sentence.lower())

        for entity in doc.ents:
            if entity.label_ == 'DATASET':
                cleaned_labels.append(clean_text(entity.text))   
            
        # CASE 1: FUZZY MATCH
        #value = fuzz.partial_ratio(sentence.lower(), known_label) # I moved .lower() here
        #if value > 85 and value < 100:
            # print('value: ', str(value), known_label) # Alex, you might wanna see what this prints
            # cleaned_labels.append(clean_text(known_label))
    
        # CASE 2: for unknown labels
        # sentence filtering (Longest Consecutive Capitalization)
        #print(sentence)
#             length, rate, filtered_sentence = LCC(sentence)
#             if rate <= 0 or length == 0 or (length == 1 and not sentence.isupper()): 
#                 continue # no consecutive caps found
#             # <insert classifier here>
#             else:
#                 for keyword in ["dataset", "data", "database", "survey", "study", "research", "statistics"]:
#                     if keyword in filtered_sentence.lower():
#                         #pass
#                         cleaned_labels.append(clean_text(filtered_sentence)) # naive
        
    cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

/home/ry/anaconda3/lib/python3.8/site-packages/spacy/util.py:1521: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
4it [00:12,  3.04s/it]


In [18]:
submission = pd.DataFrame()
submission['Id'] = id_list
submission['PredictionString'] = lables_list

In [19]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,alzheimer s disease neuroimaging initiative ad...
1,2f392438-e215-4169-bebf-21ac4ff253e1,professional development in mathematics assess...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,coastal observation station|coastal erosion st...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,high school graduate|rural urban continuum codes


In [20]:
submission.to_csv('submission.csv', index=False)

In [21]:
for item in submission["PredictionString"]:
    print(item)
    print()

alzheimer s disease neuroimaging initiative adni|adni

professional development in mathematics assessment|engineering and engineering|program focusing on research and|common core of data|school teacher salaries|and teacher professional development in mathematics and science|trends in international mathematics and science study|professional development in mathematics content|early childhood education|professional development in science assessment|nces common core of data|professional development in mathematics|higher education and lower|national center for education|program for international student

coastal observation station|coastal erosion study|coastal management and|26|coastal resources commission|sea lake and overland surges from hurricanes|slosh model|high confidence of continued landward|noaa storm surge inundation|natural resources of national|coastal change science along|high tide in|slosh display

high school graduate|rural urban continuum codes

